In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [47]:
links_df_original = pd.read_csv('simpsons_fandom_wiki_links.csv')

In [48]:
links_df = links_df_original.sort_values(['season', 'episode'], ascending=[True, True])
links_df.index = np.arange(1,len(links_df)+1)
links_df.head(10)

,title,url,episode,season
1,Simpsons Roasting on an Open Fire,https://simpsons.fandom.com/wiki/Simpsons_Roas...,1,1
2,Bart the Genius,https://simpsons.fandom.com/wiki/Bart_the_Genius,2,1
3,Homer's Odyssey,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,3,1
4,There's No Disgrace Like Home,https://simpsons.fandom.com/wiki/There%27s_No_...,4,1
5,Bart the General,https://simpsons.fandom.com/wiki/Bart_the_General,5,1
6,Moaning Lisa,https://simpsons.fandom.com/wiki/Moaning_Lisa,6,1
7,The Call of the Simpsons,https://simpsons.fandom.com/wiki/The_Call_of_t...,7,1
8,The Telltale Head,https://simpsons.fandom.com/wiki/The_Telltale_...,8,1
9,Life on the Fast Lane,https://simpsons.fandom.com/wiki/Life_on_the_F...,9,1
10,Homer's Night Out,https://simpsons.fandom.com/wiki/Homer%27s_Nig...,10,1


In [49]:
ref_page_link = 'https://simpsons.fandom.com/wiki/Simpsons_Roasting_on_an_Open_Fire/References'
req = requests.get(ref_page_link)
print(f"Request terminated with status code {req.status_code}")
print(f"Response encoded with {req.encoding}")
ref_page = BeautifulSoup(req.text, 'html.parser')

Request terminated with status code 200
Response encoded with UTF-8


In [50]:
ref_types = ['Trivia', 'Cultural references', 'Continuity', 'Goofs', 'Cultural References', 'Appearances in other media', 'Trivia/Goofs', 'Goofs/Trivia', 'Previous Episode References', 'Production Notes', 'Connections to future episodes', "Krusty's Birthday Buddies", 'Movie Moment', 'Trivia/Cultural References', 'Censorship', 'Cultural Reference', 'U.S. Syndication Cuts', 'Beatles references', 'Notes and references', 'Notes', 'Legacy', 'Running Gags:', 'Seven Deadly Sins', 'Previous and Future Episode References', 'Goofs and Continuity Errors', 'Pants Goof', 'Deleted scenes', 'DVD Release', 'Censorship and Bans', 'Reception and Legacy', 'Reception', 'Airings', 'Pranks', 'Alternate Versions', 'Call-Backs', 'External Links', 'Premiere', 'Previous episode references', 'Broadcast', 'Aerospace References', 'Awards', 'China References', 'Production', 'Goofs/errors', 'Broadcasting Information', 'Errors', 'World War II references', 'Goof', 'Notes on Known Profiles', 'Reference links', 'Songs', 'Differences from Lady and the Tramp', 'Confusion', 'References to other episodes', 'Music', 'Factual Errors', 'International premieres', 'Cuts', 'References to Toy Story in Condiments', "Goofs'", 'Continuity Errors', 'In-show references', 'Culture References', 'Characters seen in the Advent Calendar', 'Continually', 'Previous Episodes References', 'Call-Backs to Episodes (In order of appearance in the episode)']

In [51]:
# for all headers in the list find <ul> lists that may be separated with figures or smaller header
# count <li> elements in each list
new_headers = []
def get_ref_counts(episode_id):
    ref_page_link = links_df['url'].iloc[episode_id-1] + '/References'
    req = requests.get(ref_page_link)
    ref_page = BeautifulSoup(req.text, 'html.parser')
    headers = ref_page.find_all('h2')
    counter_dict = {}
    h_pointer = headers[0]
    for header in headers:
        if header.text in ref_types:
            h_pointer = header
            break

    counter_dict[h_pointer.text] = 0
    for sibling in h_pointer.next_siblings:
        if sibling.name in ['h2', 'ul']:
            # switch pointer to new header if it was encountered
            if sibling.name == 'h2':
                if sibling.text in ref_types:
                    h_pointer = sibling
                    counter_dict[h_pointer.text] = 0
                elif sibling.text == 'Call-Backs to previous episodes':
                    # hard-coded cuz it was easier to count then to write specialized parser for 1 page
                    counter_dict[sibling.text] = 31
                else:
                     if sibling.text not in new_headers:
                         new_headers.append(sibling.text)
            else:
                # count number of li elements in ul tag
                counter = 0
                for li in sibling.contents:
                    if li.name == 'li':
                        counter = counter + 1
                # increment total number of il elements of a pointed h2
                counter_dict[h_pointer.text] = counter_dict[h_pointer.text] + counter
        else:
            continue
    return counter_dict

In [52]:
counter_dicts = []
for i in range(len(links_df)):
    cd = get_ref_counts(i+1)
    counter_dicts.append(cd)

In [53]:
list_df = []
for i, cd in enumerate(counter_dicts):
    list_df.append(pd.DataFrame(cd, index=[i]))
joined_df = pd.concat(list_df)
joined_df = joined_df.fillna(0)
joined_df = joined_df.astype(int)

In [55]:
headers_0 = ['Call-Backs to Episodes (In order of appearance in the episode)', 'Rock Bottom Corrections', 'Call-Backs to previous episodes', 'Goofs in the "Italian-American-Mexican Standoff" scene', 'End Credits', 'Differences from Snow White and The Seven Dwarfs', 'Lists']

In [56]:
def check_ref_counts(episode_id):
    ref_page_link = links_df['url'].iloc[episode_id-1] + '/References'
    req = requests.get(ref_page_link)
    ref_page = BeautifulSoup(req.text, 'html.parser')
    headers = ref_page.find_all('h2')
    for header in headers:
        if header.text in headers_0:
            print(f'{header.text} : {episode_id}\n')
            break

for i in range(len(links_df)):
    cd = check_ref_counts(i+1)

Call-Backs to Episodes (In order of appearance in the episode) : 77

Rock Bottom Corrections : 112

Call-Backs to previous episodes : 286

Goofs in the "Italian-American-Mexican Standoff" scene : 313

End Credits : 360

Differences from Snow White and The Seven Dwarfs : 440

Lists : 444

Lists : 457

Lists : 504



In [66]:
links_df.loc[504]

title                                Beware My Cheating Bart
url        https://simpsons.fandom.com/wiki/Beware_My_Che...
episode                                                   18
season                                                    23
Name: 504, dtype: object

In [2]:
from fandom_wiki_refs_scraper import save_ref_counts

save_ref_counts()